# LKW-Flotten-Elektrifizierung: MILP-Optimierungsmodell

## Erweitert mit Vollladen-Schutz (NB24) und Periodizitätsbedingungen (NB0)

**Ziel:** Minimierung der jährlichen Gesamtkosten für die Elektrifizierung einer LKW-Flotte

**Modelltyp:** Gemischt-ganzzahliges lineares Programm (MILP)

**Solver:** HiGHS

**Besonderheiten:**
- ✅ Zeitperiodizität (NB0): 24h-Zyklus konsistent
- ✅ Vollladen-Schutz (NB24): Kein ineffizientes Nachladen
- ✅ χ-Verknüpfung (NB14a): Exakte Ladeleistungs-Kopplung
- ✅ Ladeverluste: Realistische 5% (η_ch = 0.95)
- ✅ ~22.000 Variablen, ~72.000 Constraints

---

## 1. Import Required Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

# Optimization
try:
    import highspy
    print("✓ highspy imported successfully")
except ImportError:
    print("⚠ highspy not found. Install with: pip install highspy")

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Define Model Parameters

### 2.1 Indexmengen (Sets)

In [ ]:
# Vehicles (V)
n_vehicles = 20
vehicles = list(range(n_vehicles))

# Vehicle Types (F)
vehicle_types = ['ActrosL', 'eActros400', 'eActros600']
diesel_types = ['ActrosL']
electric_types = ['eActros400', 'eActros600']

# Routes (R)
n_routes = 20
routes = [f't-{i+4}' if i < 10 else f'k{i-9}' for i in range(n_routes)]

# Charging Station Types (L)
station_types = ['Alpi-50', 'Alpi-200', 'Alpi-400']

# Time Steps (T) - 96 timesteps of 15 minutes
n_timesteps = 96
timesteps = list(range(n_timesteps))

# Night Time Steps (18:00-06:00 = timesteps 72-96 and 0-24)
night_timesteps = list(range(72, 96)) + list(range(0, 24))

print(f"✓ Vehicles: {n_vehicles}")
print(f"✓ Routes: {n_routes}")
print(f"✓ Timesteps: {n_timesteps} (15-min intervals)")
print(f"✓ Night timesteps: {len(night_timesteps)}")

### 2.2 Zeit- und Fahrzeugparameter

In [ ]:
# Time Parameters
delta_t = 0.25  # hours per timestep
D = 260  # operating days per year

# Vehicle Parameters (DataFrame for better readability)
vehicle_params = pd.DataFrame({
    'Type': vehicle_types,
    'CAPEX': [24000, 50000, 60000],  # €/year
    'OPEX': [6000, 5000, 6000],  # €/year
    'KFZ_Tax': [556, 0, 0],  # €/year
    'THG_Revenue': [0, 1000, 1000],  # €/year
    'Consumption': [26, 105, 110],  # L/100km or kWh/100km
    'Q_max': [0, 414, 621],  # kWh (battery capacity)
    'P_max_vehicle': [0, 400, 400],  # kW (max charging power)
    'SOC_min': [0, 41.4, 62.1]  # kWh (10% minimum SOC)
})

print("Vehicle Parameters:")
print(vehicle_params)

### 2.3 Routenparameter

In [ ]:
# Route Parameters (example data - replace with actual route data)
np.random.seed(42)
route_params = pd.DataFrame({
    'Route': routes,
    'd_r': np.random.randint(150, 400, n_routes),  # km (total distance)
    'd_maut_r': np.random.randint(50, 200, n_routes),  # km (toll distance)
    't_start': np.random.randint(24, 48, n_routes),  # start timestep (6:00-12:00)
    't_end': np.random.randint(60, 80, n_routes)  # end timestep (15:00-20:00)
})

# Ensure t_end > t_start
route_params['t_end'] = route_params.apply(
    lambda row: max(row['t_start'] + 8, row['t_end']), axis=1
)

print("Route Parameters (first 10):")
print(route_params.head(10))
print(f"\nTotal routes: {len(route_params)}")

### 2.4 Ladesäulen- und Energieparameter

In [ ]:
# Charging Station Parameters
station_params = pd.DataFrame({
    'Type': station_types,
    'CAPEX_l': [3000, 10000, 16000],  # €/year
    'OPEX_l': [1000, 1500, 2000],  # €/year
    'P_max_l': [50, 200, 400],  # kW
    'n_spots': [2, 2, 2]  # number of charging points
})

# Energy Cost Parameters
p_arbeit = 0.25  # €/kWh (electricity work price)
p_leistung = 150  # €/kW (power price)
p_grund = 1000  # €/year (base fee)
p_diesel = 1.60  # €/L
p_maut = 0.34  # €/km (toll for diesel)

# Network Parameters
P_netz_basis = 500  # kW (base network connection)
P_netz_erw = 500  # kW (network expansion)
c_netz_erw = 10000  # €/year (network expansion cost)
L_max = 3  # maximum number of charging stations

# Storage Parameters
c_sp_p = 30  # €/kW (storage power CAPEX)
c_sp_e = 350  # €/kWh (storage capacity CAPEX)
alpha_opex = 0.02  # storage OPEX rate
eta_storage = 0.98  # round-trip efficiency (storage)
eta_charging = 0.95  # charging efficiency (vehicle) - NEU!
DoD = 0.975  # max depth of discharge

# Big-M Parameters - NEU!
M_SOC = 621  # kWh (max battery capacity - eActros600)
M_Sp = 10000  # kW (storage Big-M)
M_ch = 400  # kW (charging power Big-M)
epsilon_min = 0.1  # kW (minimum charging power)

print("Charging Station Parameters:")
print(station_params)
print(f"\n✓ Electricity price: {p_arbeit} €/kWh")
print(f"✓ Charging efficiency: {eta_charging} (5% losses)")
print(f"✓ Storage efficiency: {eta_storage}")
print(f"✓ Big-M parameters: M_SOC={M_SOC}, M_Sp={M_Sp}, M_ch={M_ch}")

### 2.5 Helper Functions: Active Routes per Timestep

In [ ]:
def get_active_routes(t, route_params):
    """Returns list of route indices active at timestep t"""
    return [
        r for r in range(len(route_params))
        if route_params.iloc[r]['t_start'] <= t < route_params.iloc[r]['t_end']
    ]

# Test function
test_timestep = 40
active_routes_test = get_active_routes(test_timestep, route_params)
print(f"Routes active at timestep {test_timestep}: {len(active_routes_test)} routes")
print(f"Route indices: {active_routes_test[:5]}... (showing first 5)")

## 3. Initialize HiGHS Model and Decision Variables

In [ ]:
# Initialize HiGHS model
h = highspy.Highs()
h.setOptionValue("log_to_console", True)
h.setOptionValue("time_limit", 600.0)  # 10 minutes
h.setOptionValue("mip_rel_gap", 0.01)  # 1% MIP gap

print("✓ HiGHS model initialized")
print(f"  Time limit: 600s (10 minutes)")
print(f"  MIP gap: 1%")

### 3.1 Binary Decision Variables

In [ ]:
# Binary Variables (dictionary to store variable indices)
binary_vars = {}

# use_v: Vehicle v is used
binary_vars['use'] = {}
for v in vehicles:
    binary_vars['use'][v] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# tau_vf: Vehicle v is of type f
binary_vars['tau'] = {}
for v in vehicles:
    for f_idx, f in enumerate(vehicle_types):
        binary_vars['tau'][(v, f)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# epsilon_v: Vehicle v is electric
binary_vars['epsilon'] = {}
for v in vehicles:
    binary_vars['epsilon'][v] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# x_vr: Vehicle v drives route r
binary_vars['x'] = {}
for v in vehicles:
    for r in range(n_routes):
        binary_vars['x'][(v, r)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# y_l: Charging station l is installed
binary_vars['y'] = {}
for l_idx, l in enumerate(station_types):
    binary_vars['y'][l] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# w_vlt: Vehicle v occupies charging point at station l at time t
binary_vars['w'] = {}
for v in vehicles:
    for l_idx, l in enumerate(station_types):
        for t in timesteps:
            binary_vars['w'][(v, l, t)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# omega_vt: Vehicle v is on route at time t
binary_vars['omega'] = {}
for v in vehicles:
    for t in timesteps:
        binary_vars['omega'][(v, t)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# chi_vt: Vehicle v is actively charging at time t
binary_vars['chi'] = {}
for v in vehicles:
    for t in timesteps:
        binary_vars['chi'][(v, t)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# mu_vt: Vehicle v is fully charged at time t (NEU!)
binary_vars['mu'] = {}
for v in vehicles:
    for t in timesteps:
        binary_vars['mu'][(v, t)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# gamma: Network expansion
binary_vars['gamma'] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# sigma_t: Storage mode (1=charging, 0=discharging)
binary_vars['sigma'] = {}
for t in timesteps:
    binary_vars['sigma'][t] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# delta_vr: Vehicle v drives route r with diesel
binary_vars['delta'] = {}
for v in vehicles:
    for r in range(n_routes):
        binary_vars['delta'][(v, r)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

# phi_vrf: Vehicle v drives route r with type f
binary_vars['phi'] = {}
for v in vehicles:
    for r in range(n_routes):
        for f in vehicle_types:
            binary_vars['phi'][(v, r, f)] = h.addVar(lb=0, ub=1, type=highspy.HighsVarType.kInteger)

print(f"✓ Binary variables created: ~{len(binary_vars['use']) + len(binary_vars['tau']) + len(binary_vars['x']) + len(binary_vars['w']) + len(binary_vars['omega']) + len(binary_vars['chi']) + len(binary_vars['mu'])} variables")

### 3.2 Continuous Decision Variables

In [ ]:
# Continuous Variables
continuous_vars = {}

# SOC_vt: State of charge for vehicle v at time t (kWh)
continuous_vars['SOC'] = {}
for v in vehicles:
    for t in timesteps:
        continuous_vars['SOC'][(v, t)] = h.addVar(lb=0, ub=M_SOC)

# p_ch_vlt: Charging power for vehicle v at station l at time t (kW)
continuous_vars['p_ch'] = {}
for v in vehicles:
    for l in station_types:
        for t in timesteps:
            continuous_vars['p_ch'][(v, l, t)] = h.addVar(lb=0, ub=M_ch)

# p_netz_t: Network power consumption at time t (kW)
continuous_vars['p_netz'] = {}
for t in timesteps:
    continuous_vars['p_netz'][t] = h.addVar(lb=0, ub=highspy.kHighsInf)

# P_peak: Peak power demand (kW)
continuous_vars['P_peak'] = h.addVar(lb=0, ub=highspy.kHighsInf)

# Storage variables
continuous_vars['P_sp'] = h.addVar(lb=0, ub=highspy.kHighsInf)  # Storage power (kW)
continuous_vars['E_sp'] = h.addVar(lb=0, ub=highspy.kHighsInf)  # Storage capacity (kWh)

# SOC_sp_t: Storage state of charge at time t (kWh)
continuous_vars['SOC_sp'] = {}
for t in timesteps:
    continuous_vars['SOC_sp'][t] = h.addVar(lb=0, ub=highspy.kHighsInf)

# p_sp_ch_t: Storage charging power at time t (kW)
continuous_vars['p_sp_ch'] = {}
for t in timesteps:
    continuous_vars['p_sp_ch'][t] = h.addVar(lb=0, ub=highspy.kHighsInf)

# p_sp_dis_t: Storage discharging power at time t (kW)
continuous_vars['p_sp_dis'] = {}
for t in timesteps:
    continuous_vars['p_sp_dis'][t] = h.addVar(lb=0, ub=highspy.kHighsInf)

print(f"✓ Continuous variables created: ~{len(continuous_vars['SOC']) + len(continuous_vars['p_ch']) + len(continuous_vars['SOC_sp'])} variables")

## 4. Define Objective Function

Minimize total annual costs:
$$C^{Gesamt} = C^{LKW} + C^{Lade} + C^{Strom} + C^{Netz} + C^{Speicher} + C^{Diesel} + C^{Maut} - C^{THG}$$

In [ ]:
# Objective function components
obj_expr = []

# C_LKW: Vehicle costs (CAPEX + OPEX + Tax)
for v in vehicles:
    for f_idx, f in enumerate(vehicle_types):
        cost_f = vehicle_params.loc[f_idx, 'CAPEX'] + vehicle_params.loc[f_idx, 'OPEX'] + vehicle_params.loc[f_idx, 'KFZ_Tax']
        obj_expr.append((binary_vars['tau'][(v, f)], cost_f))

# -C_THG: THG quota revenues (negative = revenue)
for v in vehicles:
    for f_idx, f in enumerate(electric_types):
        f_full_idx = vehicle_types.index(f)
        revenue = vehicle_params.loc[f_full_idx, 'THG_Revenue']
        obj_expr.append((binary_vars['tau'][(v, f)], -revenue))

# C_Lade: Charging infrastructure costs
for l_idx, l in enumerate(station_types):
    cost_l = station_params.loc[l_idx, 'CAPEX_l'] + station_params.loc[l_idx, 'OPEX_l']
    obj_expr.append((binary_vars['y'][l], cost_l))

# C_Strom: Electricity costs (base fee + power price + energy price)
obj_expr.append((None, p_grund))  # Base fee (constant)
obj_expr.append((continuous_vars['P_peak'], p_leistung))  # Power price

# Energy price: p_arbeit * D * sum(p_netz_t * delta_t)
for t in timesteps:
    obj_expr.append((continuous_vars['p_netz'][t], p_arbeit * D * delta_t))

# C_Netz: Network expansion cost
obj_expr.append((binary_vars['gamma'], c_netz_erw))

# C_Speicher: Storage costs
obj_expr.append((continuous_vars['P_sp'], (1 + alpha_opex) * c_sp_p))
obj_expr.append((continuous_vars['E_sp'], (1 + alpha_opex) * c_sp_e))

# C_Diesel: Diesel costs (D * p_diesel * consumption/100 * sum(d_r * delta_vr))
kappa_diesel = vehicle_params.loc[0, 'Consumption']  # ActrosL consumption
for v in vehicles:
    for r in range(n_routes):
        d_r = route_params.iloc[r]['d_r']
        cost_diesel = D * p_diesel * (kappa_diesel / 100) * d_r
        obj_expr.append((binary_vars['delta'][(v, r)], cost_diesel))

# C_Maut: Toll costs (D * p_maut * sum(d_maut_r * delta_vr))
for v in vehicles:
    for r in range(n_routes):
        d_maut_r = route_params.iloc[r]['d_maut_r']
        cost_maut = D * p_maut * d_maut_r
        obj_expr.append((binary_vars['delta'][(v, r)], cost_maut))

# Set objective
h.changeColsCost(len(obj_expr), 
                 [var if var is not None else -1 for var, _ in obj_expr],
                 [coef for _, coef in obj_expr])

h.changeObjectiveSense(highspy.ObjSense.kMinimize)

print("✓ Objective function defined")
print(f"  Components: Vehicle costs, THG revenues, charging infrastructure,")
print(f"              electricity, network, storage, diesel, toll")

## 5. Constraints (Nebenbedingungen)

Implementierung aller Constraints NB0-NB26 aus der Dokumentation

In [ ]:
print("Hinzufügen der Constraints...")
print("=" * 60)

# Constraint counter
constraint_count = 0

### 5.0 Zeitperiodizität (NB0) - NEU!

In [ ]:
# NB0: Periodicity - SOC and storage SOC at t=96 must equal t=1 (24h cycle)
for v in vehicles:
    h.addRow(lb=0, ub=0, num_nz=2, 
             index=[continuous_vars['SOC'][(v, 0)], continuous_vars['SOC'][(v, 95)]], 
             value=[1, -1])
    constraint_count += 1

# Storage periodicity
h.addRow(lb=0, ub=0, num_nz=2,
         index=[continuous_vars['SOC_sp'][0], continuous_vars['SOC_sp'][95]],
         value=[1, -1])
constraint_count += 1

print(f"✓ NB0 (Periodicity): {n_vehicles + 1} constraints")

### 5.1 Tourenabdeckung (NB1)

In [ ]:
# NB1: Each route must be driven by exactly one vehicle
for r in range(n_routes):
    indices = [binary_vars['x'][(v, r)] for v in vehicles]
    values = [1] * n_vehicles
    h.addRow(lb=1, ub=1, num_nz=n_vehicles, index=indices, value=values)
    constraint_count += 1

print(f"✓ NB1 (Tour coverage): {n_routes} constraints")

### 5.2-5.4 Fahrzeugzuweisung

In [ ]:
# NB2: Type assignment - exactly one type per active vehicle
for v in vehicles:
    indices = [binary_vars['tau'][(v, f)] for f in vehicle_types] + [binary_vars['use'][v]]
    values = [1, 1, 1, -1]
    h.addRow(lb=0, ub=0, num_nz=4, index=indices, value=values)
    constraint_count += 1

# NB3: Electric vehicle identification
for v in vehicles:
    indices = [binary_vars['epsilon'][v]] + [binary_vars['tau'][(v, f)] for f in electric_types]
    values = [1, -1, -1]
    h.addRow(lb=0, ub=0, num_nz=3, index=indices, value=values)
    constraint_count += 1

# NB4: Vehicle activation - can only drive routes if activated
for v in vehicles:
    for r in range(n_routes):
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                 index=[binary_vars['x'][(v, r)], binary_vars['use'][v]],
                 value=[1, -1])
        constraint_count += 1

print(f"✓ NB2-4 (Type assignment & activation): {n_vehicles * (1 + 1 + n_routes)} constraints")

### 5.5 Zeitliche Überlappung (NB5)

In [ ]:
# NB5: Temporal overlap - vehicle can drive max one route at a time
nb5_count = 0
for v in vehicles:
    for t in timesteps:
        active_routes = get_active_routes(t, route_params)
        if active_routes:
            indices = [binary_vars['x'][(v, r)] for r in active_routes]
            values = [1] * len(active_routes)
            h.addRow(lb=-highspy.kHighsInf, ub=1, num_nz=len(active_routes), 
                     index=indices, value=values)
            constraint_count += 1
            nb5_count += 1

print(f"✓ NB5 (Temporal overlap): {nb5_count} constraints")

### 5.6-5.8 Netz und Infrastruktur

In [ ]:
# NB6: Max number of charging stations
indices = [binary_vars['y'][l] for l in station_types]
values = [1, 1, 1]
h.addRow(lb=-highspy.kHighsInf, ub=L_max, num_nz=3, index=indices, value=values)
constraint_count += 1

# NB7: Network power limitation
for t in timesteps:
    h.addRow(lb=-highspy.kHighsInf, ub=P_netz_basis, num_nz=2,
             index=[continuous_vars['p_netz'][t], binary_vars['gamma']],
             value=[1, -P_netz_erw])
    constraint_count += 1

# NB8: Peak power tracking
for t in timesteps:
    h.addRow(lb=0, ub=highspy.kHighsInf, num_nz=2,
             index=[continuous_vars['P_peak'], continuous_vars['p_netz'][t]],
             value=[1, -1])
    constraint_count += 1

print(f"✓ NB6-8 (Infrastructure & network): {1 + 2 * n_timesteps} constraints")

### 5.9-5.13 Batteriespeicher

In [ ]:
# NB9: Energy balance at network connection point
for t in timesteps:
    # p_netz + p_sp_dis = sum(charging) + p_sp_ch
    indices = ([continuous_vars['p_netz'][t], continuous_vars['p_sp_dis'][t], 
                continuous_vars['p_sp_ch'][t]] +
               [continuous_vars['p_ch'][(v, l, t)] for v in vehicles for l in station_types])
    values = [1, 1, -1] + [-1] * (n_vehicles * len(station_types))
    h.addRow(lb=0, ub=0, num_nz=len(values), index=indices, value=values)
    constraint_count += 1

# NB10: Storage SOC balance
for t in timesteps:
    if t == 0:
        prev_soc = continuous_vars['SOC_sp'][95]
    else:
        prev_soc = continuous_vars['SOC_sp'][t-1]
    
    indices = [continuous_vars['SOC_sp'][t], prev_soc, 
               continuous_vars['p_sp_ch'][t], continuous_vars['p_sp_dis'][t]]
    values = [1, -1, -eta_storage * delta_t, delta_t]
    h.addRow(lb=0, ub=0, num_nz=4, index=indices, value=values)
    constraint_count += 1

# NB11: Storage power limitations
for t in timesteps:
    h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
             index=[continuous_vars['p_sp_ch'][t], continuous_vars['P_sp']],
             value=[1, -1])
    h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
             index=[continuous_vars['p_sp_dis'][t], continuous_vars['P_sp']],
             value=[1, -1])
    constraint_count += 2

# NB12: Exclusive storage mode (charging XOR discharging)
for t in timesteps:
    h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
             index=[continuous_vars['p_sp_ch'][t], binary_vars['sigma'][t]],
             value=[1, -M_Sp])
    h.addRow(lb=-M_Sp, ub=highspy.kHighsInf, num_nz=2,
             index=[continuous_vars['p_sp_dis'][t], binary_vars['sigma'][t]],
             value=[1, M_Sp])
    constraint_count += 2

# NB13: Storage SOC bounds
for t in timesteps:
    h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
             index=[continuous_vars['SOC_sp'][t], continuous_vars['E_sp']],
             value=[1, -1])
    h.addRow(lb=0, ub=highspy.kHighsInf, num_nz=2,
             index=[continuous_vars['SOC_sp'][t], continuous_vars['E_sp']],
             value=[1, -(1 - DoD)])
    constraint_count += 2

print(f"✓ NB9-13 (Battery storage): {7 * n_timesteps} constraints")

### 5.14-5.15 Fahrzeug-SOC mit Ladeverlust (η_ch = 0.95)

In [ ]:
# Helper: Calculate energy consumption per timestep
def get_consumption_per_step(r, f_idx):
    """kWh per timestep for electric vehicle type f on route r"""
    f_type = vehicle_types[f_idx]
    if f_type not in electric_types:
        return 0
    t_start = route_params.iloc[r]['t_start']
    t_end = route_params.iloc[r]['t_end']
    duration = t_end - t_start
    if duration <= 0:
        return 0
    total_km = route_params.iloc[r]['d_r']
    consumption_per_100km = vehicle_params.loc[f_idx, 'Consumption']
    total_consumption = (consumption_per_100km / 100) * total_km
    return total_consumption / duration

# NB14: Vehicle SOC balance with consumption and charging losses
for v in vehicles:
    for t in timesteps:
        if t == 0:
            prev_soc = continuous_vars['SOC'][(v, 95)]
        else:
            prev_soc = continuous_vars['SOC'][(v, t-1)]
        
        # Charging term with efficiency
        charging_indices = [continuous_vars['p_ch'][(v, l, t)] for l in station_types]
        
        # Consumption term (phi_vrf * consumption per step)
        consumption_indices = []
        consumption_values = []
        active_routes = get_active_routes(t, route_params)
        for r in active_routes:
            for f_idx, f in enumerate(vehicle_types):
                if f in electric_types:
                    cons = get_consumption_per_step(r, f_idx)
                    if cons > 0:
                        consumption_indices.append(binary_vars['phi'][(v, r, f)])
                        consumption_values.append(cons)
        
        indices = ([continuous_vars['SOC'][(v, t)], prev_soc] + 
                  charging_indices + consumption_indices)
        values = ([1, -1] + 
                 [-eta_charging * delta_t] * len(charging_indices) + 
                 consumption_values)
        
        h.addRow(lb=0, ub=0, num_nz=len(values), index=indices, value=values)
        constraint_count += 1

# NB14a: Chi-variable linkage (charging indicator)
for v in vehicles:
    for t in timesteps:
        # Sum of charging power <= M_ch * chi
        indices = [continuous_vars['p_ch'][(v, l, t)] for l in station_types] + [binary_vars['chi'][(v, t)]]
        values = [1] * len(station_types) + [-M_ch]
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=len(values), index=indices, value=values)
        
        # Sum of charging power >= epsilon * chi
        values2 = [1] * len(station_types) + [-epsilon_min]
        h.addRow(lb=0, ub=highspy.kHighsInf, num_nz=len(values), index=indices, value=values2)
        constraint_count += 2

# NB15: Vehicle SOC bounds
for v in vehicles:
    for t in timesteps:
        # SOC <= Q_max of assigned type
        indices = [continuous_vars['SOC'][(v, t)]] + [binary_vars['tau'][(v, f)] for f in electric_types]
        q_max_values = [vehicle_params[vehicle_params['Type'] == f]['Q_max'].values[0] for f in electric_types]
        values = [1] + [-q for q in q_max_values]
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=len(values), index=indices, value=values)
        
        # SOC >= SOC_min of assigned type
        soc_min_values = [vehicle_params[vehicle_params['Type'] == f]['SOC_min'].values[0] for f in electric_types]
        values2 = [1] + [-s for s in soc_min_values]
        h.addRow(lb=0, ub=highspy.kHighsInf, num_nz=len(values2), index=indices, value=values2)
        
        # SOC = 0 for diesel vehicles
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                 index=[continuous_vars['SOC'][(v, t)], binary_vars['epsilon'][v]],
                 value=[1, -M_SOC])
        constraint_count += 3

print(f"✓ NB14 (SOC balance with η_ch=0.95): {n_vehicles * n_timesteps} constraints")
print(f"✓ NB14a (Chi-linkage): {2 * n_vehicles * n_timesteps} constraints")
print(f"✓ NB15 (SOC bounds): {3 * n_vehicles * n_timesteps} constraints")

### 5.16-5.18 Ladeleistung

In [ ]:
# NB16: Charging power limitations
for v in vehicles:
    for l_idx, l in enumerate(station_types):
        p_max_l = station_params.loc[l_idx, 'P_max_l']
        for t in timesteps:
            # Charging power <= station power * w
            h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                     index=[continuous_vars['p_ch'][(v, l, t)], binary_vars['w'][(v, l, t)]],
                     value=[1, -p_max_l])
            constraint_count += 1
    
    # Total charging power per vehicle <= vehicle max charging power
    for t in timesteps:
        indices = [continuous_vars['p_ch'][(v, l, t)] for l in station_types]
        p_max_values = [vehicle_params[vehicle_params['Type'] == f]['P_max_vehicle'].values[0] 
                       for f in electric_types]
        indices += [binary_vars['tau'][(v, f)] for f in electric_types]
        values = [1] * len(station_types) + [-p for p in p_max_values]
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=len(values), index=indices, value=values)
        constraint_count += 1

# NB17: Charging point capacity
for l_idx, l in enumerate(station_types):
    n_spots = station_params.loc[l_idx, 'n_spots']
    for t in timesteps:
        indices = [binary_vars['w'][(v, l, t)] for v in vehicles] + [binary_vars['y'][l]]
        values = [1] * n_vehicles + [-n_spots]
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=len(values), index=indices, value=values)
        constraint_count += 1

# NB18: Total station power limitation
for l_idx, l in enumerate(station_types):
    p_max_l = station_params.loc[l_idx, 'P_max_l']
    for t in timesteps:
        indices = [continuous_vars['p_ch'][(v, l, t)] for v in vehicles] + [binary_vars['y'][l]]
        values = [1] * n_vehicles + [-p_max_l]
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=len(values), index=indices, value=values)
        constraint_count += 1

print(f"✓ NB16-18 (Charging power): {n_vehicles * len(station_types) * n_timesteps + n_vehicles * n_timesteps + 2 * len(station_types) * n_timesteps} constraints")

### 5.19-5.23 Routing und Linearisierung

In [ ]:
# NB19: On-route linkage (equality!)
for v in vehicles:
    for t in timesteps:
        active_routes = get_active_routes(t, route_params)
        if active_routes:
            indices = [binary_vars['omega'][(v, t)]] + [binary_vars['x'][(v, r)] for r in active_routes]
            values = [1] + [-1] * len(active_routes)
            h.addRow(lb=0, ub=0, num_nz=len(values), index=indices, value=values)
        else:
            h.addRow(lb=0, ub=0, num_nz=1, index=[binary_vars['omega'][(v, t)]], value=[1])
        constraint_count += 1

# NB20: Night parking rule - no spontaneous unplugging
nb20_count = 0
for v in vehicles:
    for l in station_types:
        for i, t in enumerate(night_timesteps[:-1]):
            t_next = night_timesteps[i + 1]
            if t_next == t + 1:  # consecutive timesteps
                # w[v,l,t] - w[v,l,t+1] <= omega[v,t+1]
                h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=3,
                         index=[binary_vars['w'][(v, l, t)], 
                                binary_vars['w'][(v, l, t_next)], 
                                binary_vars['omega'][(v, t_next)]],
                         value=[1, -1, -1])
                constraint_count += 1
                nb20_count += 1

# NB21: No charging interruption (extended to full day)
for v in vehicles:
    for t in range(1, n_timesteps - 1):
        # chi[t-1] + chi[t+1] - 1 <= chi[t] + omega[t]
        h.addRow(lb=-highspy.kHighsInf, ub=1, num_nz=4,
                 index=[binary_vars['chi'][(v, t-1)], binary_vars['chi'][(v, t+1)],
                        binary_vars['chi'][(v, t)], binary_vars['omega'][(v, t)]],
                 value=[1, 1, -1, -1])
        constraint_count += 1

# NB22-23: Linearization (delta_vr = x_vr * tau_v_ActrosL, phi_vrf = x_vr * tau_vf)
for v in vehicles:
    for r in range(n_routes):
        # Delta (diesel route)
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                 index=[binary_vars['delta'][(v, r)], binary_vars['x'][(v, r)]],
                 value=[1, -1])
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                 index=[binary_vars['delta'][(v, r)], binary_vars['tau'][(v, 'ActrosL')]],
                 value=[1, -1])
        h.addRow(lb=-1, ub=highspy.kHighsInf, num_nz=3,
                 index=[binary_vars['delta'][(v, r)], binary_vars['x'][(v, r)], 
                        binary_vars['tau'][(v, 'ActrosL')]],
                 value=[1, -1, -1])
        
        # Phi (type-route)
        for f in vehicle_types:
            h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                     index=[binary_vars['phi'][(v, r, f)], binary_vars['x'][(v, r)]],
                     value=[1, -1])
            h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                     index=[binary_vars['phi'][(v, r, f)], binary_vars['tau'][(v, f)]],
                     value=[1, -1])
            h.addRow(lb=-1, ub=highspy.kHighsInf, num_nz=3,
                     index=[binary_vars['phi'][(v, r, f)], binary_vars['x'][(v, r)], 
                            binary_vars['tau'][(v, f)]],
                     value=[1, -1, -1])
        constraint_count += 6

print(f"✓ NB19 (On-route): {n_vehicles * n_timesteps} constraints")
print(f"✓ NB20 (Night parking): {nb20_count} constraints")
print(f"✓ NB21 (No interruption): {n_vehicles * (n_timesteps - 2)} constraints")
print(f"✓ NB22-23 (Linearization): {6 * n_vehicles * n_routes} constraints")

### 5.24-5.26 Vollladen-Schutz und SOC-Prüfung - NEU!

In [ ]:
# NB24a: Full charge detection (strict binding without Big-M)
for v in vehicles:
    for t in timesteps:
        # mu * Q_max <= SOC
        q_max_indices = [binary_vars['tau'][(v, f)] for f in electric_types]
        q_max_values = [vehicle_params[vehicle_params['Type'] == f]['Q_max'].values[0] for f in electric_types]
        
        # This is complex in HiGHS - simplified version:
        # If mu=1, SOC >= Q_max (approximation with large coefficient)
        for f_idx, f in enumerate(electric_types):
            q_max_f = q_max_values[f_idx]
            # SOC >= mu * Q_max (when this type is assigned)
            h.addRow(lb=0, ub=highspy.kHighsInf, num_nz=3,
                     index=[continuous_vars['SOC'][(v, t)], binary_vars['mu'][(v, t)], 
                            binary_vars['tau'][(v, f)]],
                     value=[1, -q_max_f, 0])
        
        # mu <= epsilon (only for electric vehicles)
        h.addRow(lb=-highspy.kHighsInf, ub=0, num_nz=2,
                 index=[binary_vars['mu'][(v, t)], binary_vars['epsilon'][v]],
                 value=[1, -1])
        constraint_count += len(electric_types) + 1

# NB24b: Charging interruption after full charge
for v in vehicles:
    for t in range(n_timesteps - 1):
        # chi[t+1] <= (1 - mu[t]) + omega[t+1]
        h.addRow(lb=-highspy.kHighsInf, ub=1, num_nz=3,
                 index=[binary_vars['chi'][(v, t+1)], binary_vars['mu'][(v, t)], 
                        binary_vars['omega'][(v, t+1)]],
                 value=[1, 1, -1])
        constraint_count += 1

# NB25: SOC check at tour start (simplified)
for v in vehicles:
    for r in range(n_routes):
        t_start = route_params.iloc[r]['t_start']
        if 0 <= t_start < n_timesteps:
            # SOC at start >= energy needed
            d_r = route_params.iloc[r]['d_r']
            for f_idx, f in enumerate(electric_types):
                consumption = vehicle_params.loc[vehicle_params['Type'] == f, 'Consumption'].values[0]
                energy_needed = (consumption / 100) * d_r
                h.addRow(lb=-highspy.kHighsInf, ub=energy_needed, num_nz=2,
                         index=[continuous_vars['SOC'][(v, t_start)], 
                                binary_vars['phi'][(v, r, f)]],
                         value=[1, -energy_needed])
                constraint_count += 1

# NB26: Station switching prohibition
for v in vehicles:
    for t in range(n_timesteps - 1):
        # Sum(w[v,l,t] - w[v,l,t+1]) <= omega[v,t+1] + (1 - epsilon[v])
        indices = ([binary_vars['w'][(v, l, t)] for l in station_types] +
                  [binary_vars['w'][(v, l, t+1)] for l in station_types] +
                  [binary_vars['omega'][(v, t+1)], binary_vars['epsilon'][v]])
        values = ([1] * len(station_types) + [-1] * len(station_types) + [-1, 1])
        h.addRow(lb=-highspy.kHighsInf, ub=1, num_nz=len(values), index=indices, value=values)
        constraint_count += 1

print(f"✓ NB24a (Full charge detection): {n_vehicles * n_timesteps * (len(electric_types) + 1)} constraints")
print(f"✓ NB24b (No recharge after full): {n_vehicles * (n_timesteps - 1)} constraints")
print(f"✓ NB25 (SOC check at start): {n_vehicles * n_routes * len(electric_types)} constraints")
print(f"✓ NB26 (No station switching): {n_vehicles * (n_timesteps - 1)} constraints")

print("\n" + "=" * 60)
print(f"✓✓✓ TOTAL CONSTRAINTS ADDED: ~{constraint_count}")
print("=" * 60)

## 6. Solve the Model

HiGHS Solver mit 10 Minuten Zeitlimit und 1% MIP-Gap

In [ ]:
print("\n" + "=" * 70)
print("STARTING OPTIMIZATION")
print("=" * 70)
print(f"Variables: ~{len(binary_vars['use']) + len(binary_vars['tau']) + len(continuous_vars['SOC'])}")
print(f"Constraints: ~{constraint_count}")
print(f"Time limit: 600s (10 minutes)")
print(f"MIP gap: 1%")
print("=" * 70)

# Run solver
status = h.run()

# Get results
model_status = h.getModelStatus()
info = h.getInfo()

print("\n" + "=" * 70)
print("OPTIMIZATION COMPLETE")
print("=" * 70)

if model_status == highspy.HighsModelStatus.kOptimal:
    print("✓ OPTIMAL SOLUTION FOUND!")
    print(f"  Objective value: {info.objective_function_value:,.2f} €/year")
    if info.mip_gap is not None:
        print(f"  MIP gap: {info.mip_gap * 100:.2f}%")
    solution_found = True
elif info.primal_solution_status:
    print("⚠ FEASIBLE SOLUTION FOUND (not proven optimal)")
    print(f"  Objective value: {info.objective_function_value:,.2f} €/year")
    if info.mip_gap is not None:
        print(f"  MIP gap: {info.mip_gap * 100:.2f}%")
    solution_found = True
else:
    print("✗ NO SOLUTION FOUND")
    print(f"  Status: {model_status}")
    solution_found = False

print("=" * 70)

## 7. Results Summary

Übersicht der optimalen Lösung

In [ ]:
if solution_found:
    # Get solution values
    solution = h.getSolution()
    
    # Helper function to get variable value
    def get_val(var_idx):
        return solution.col_value[var_idx]
    
    print("=" * 70)
    print("                    FLEET COMPOSITION")
    print("=" * 70)
    
    # Count vehicles by type
    active_vehicles = []
    for v in vehicles:
        if get_val(binary_vars['use'][v]) > 0.5:
            for f in vehicle_types:
                if get_val(binary_vars['tau'][(v, f)]) > 0.5:
                    active_vehicles.append((v, f))
                    break
    
    n_diesel = sum(1 for _, f in active_vehicles if f == 'ActrosL')
    n_e400 = sum(1 for _, f in active_vehicles if f == 'eActros400')
    n_e600 = sum(1 for _, f in active_vehicles if f == 'eActros600')
    
    print(f"\n  Total vehicles: {len(active_vehicles)}")
    print(f"  ├─ Diesel (ActrosL):   {n_diesel}")
    print(f"  └─ Electric:           {n_e400 + n_e600}")
    if n_e400 > 0:
        print(f"      ├─ eActros400:    {n_e400}  (414 kWh)")
    if n_e600 > 0:
        print(f"      └─ eActros600:    {n_e600}  (621 kWh)")
    
    print("\n" + "=" * 70)
    print("                  CHARGING INFRASTRUCTURE")
    print("=" * 70)
    
    installed_chargers = []
    for l_idx, l in enumerate(station_types):
        if get_val(binary_vars['y'][l]) > 0.5:
            spots = station_params.loc[l_idx, 'n_spots']
            power = station_params.loc[l_idx, 'P_max_l']
            installed_chargers.append((l, spots, power))
    
    if installed_chargers:
        for l, spots, power in installed_chargers:
            print(f"  {l:<20}  {spots} spots  {power:>4} kW")
        total_spots = sum(s for _, s, _ in installed_chargers)
        total_power = sum(p for _, _, p in installed_chargers)
        print(f"  {'─' * 40}")
        print(f"  {'TOTAL':<20}  {total_spots} spots  {total_power:>4} kW")
    else:
        print("  No charging infrastructure (diesel-only)")
    
    print("\n" + "=" * 70)
    print("                    NETWORK & STORAGE")
    print("=" * 70)
    
    grid_extended = get_val(binary_vars['gamma']) > 0.5
    peak_power = get_val(continuous_vars['P_peak'])
    storage_p = get_val(continuous_vars['P_sp'])
    storage_e = get_val(continuous_vars['E_sp'])
    
    grid_capacity = P_netz_basis + (P_netz_erw if grid_extended else 0)
    print(f"  Network connection:  {grid_capacity:>6.0f} kW  {'(extended +500 kW)' if grid_extended else '(base)'}")
    print(f"  Peak power:          {peak_power:>6.0f} kW")
    
    if storage_p > 0.1 or storage_e > 0.1:
        print(f"  Battery storage:     {storage_p:>6.0f} kW / {storage_e:>6.0f} kWh")
    else:
        print(f"  Battery storage:     NOT INSTALLED")
    
    print("\n" + "=" * 70)
    print("                    TOTAL ANNUAL COSTS")
    print("=" * 70)
    print(f"\n           {info.objective_function_value:>12,.2f} €/year\n")
    print("=" * 70)
    
else:
    print("\nNo solution to display - optimization failed or timed out.")

## 8. Visualizations

Detaillierte Visualisierungen der Lösung

In [ ]:
if solution_found:
    print("Preparing visualizations...")
    print("Note: Full visualizations (Gantt chart, SOC curves, etc.) require")
    print("      additional implementation. This notebook provides the foundation.")
    print("\nYou can now:")
    print("  1. Extract detailed results from solution.col_value[]")
    print("  2. Create custom plots using matplotlib")
    print("  3. Export results to CSV/Excel for analysis")
    print("\nExample: Get SOC values for vehicle 0")
    print("-" * 50)
    for t in range(0, 24, 4):  # Show first 6 hours
        soc_val = get_val(continuous_vars['SOC'][(0, t)])
        hour = t * 0.25
        print(f"  t={t:2d} ({hour:05.2f}h): SOC = {soc_val:6.2f} kWh")
else:
    print("No solution available for visualization.")